In [0]:
#!pip install opencv-python
#!pip install imutils

import cv2
import numpy as np
import dlib
import imutils
from google.colab.patches import cv2_imshow
from imutils.video import FileVideoStream
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = (6.0,6.0)
matplotlib.rcParams['image.cmap'] = 'gray'


In [2]:
from google.colab import drive

drive.mount('/content/drive')
!ls /content/drive/My\ Drive/EVA/Assignment23 > /dev/null

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls /content/drive/My\ Drive/EVA/Assignment23/VID_20191020_132419.mp4
!ls /content/drive/My\ Drive/EVA/Assignment23/data/

'/content/drive/My Drive/EVA/Assignment23/VID_20191020_132419.mp4'
frame0.jpg    frame121.jpg  frame14.jpg  frame36.jpg  frame58.jpg  frame7.jpg
frame100.jpg  frame122.jpg  frame15.jpg  frame37.jpg  frame59.jpg  frame80.jpg
frame101.jpg  frame123.jpg  frame16.jpg  frame38.jpg  frame5.jpg   frame81.jpg
frame102.jpg  frame124.jpg  frame17.jpg  frame39.jpg  frame60.jpg  frame82.jpg
frame103.jpg  frame125.jpg  frame18.jpg  frame3.jpg   frame61.jpg  frame83.jpg
frame104.jpg  frame126.jpg  frame19.jpg  frame40.jpg  frame62.jpg  frame84.jpg
frame105.jpg  frame127.jpg  frame1.jpg	 frame41.jpg  frame63.jpg  frame85.jpg
frame106.jpg  frame128.jpg  frame20.jpg  frame42.jpg  frame64.jpg  frame86.jpg
frame107.jpg  frame129.jpg  frame21.jpg  frame43.jpg  frame65.jpg  frame87.jpg
frame108.jpg  frame12.jpg   frame22.jpg  frame44.jpg  frame66.jpg  frame88.jpg
frame109.jpg  frame130.jpg  frame23.jpg  frame45.jpg  frame67.jpg  frame89.jpg
frame10.jpg   frame131.jpg  frame24.jpg  frame46.jpg  frame68.jpg 

In [4]:
MODEL_PATH="/content/drive/My Drive/EVA/Assignment23/"
PREDICTOR_PATH = MODEL_PATH + "shape_predictor_68_face_landmarks.dat"
DATA_PATH=MODEL_PATH + "data/"
OUTPUTPATH=MODEL_PATH + "LandmarksOutPut/"
LANDMARKSPATH=MODEL_PATH + "Landmarks/"

srcframePrefix = "frame"
srcframeSuffix = ".jpg"

print(PREDICTOR_PATH)
print(DATA_PATH)
print(OUTPUTPATH)


/content/drive/My Drive/EVA/Assignment23/shape_predictor_68_face_landmarks.dat
/content/drive/My Drive/EVA/Assignment23/data/
/content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/


In [0]:
def detectFace(im):
  # Detect faces in the image
  faceRects = faceDetector(im, 0)
  #print("Number of faces detected: ",len(faceRects))

  return faceRects

In [0]:

def drawPolyline(im, landmarks, start, end, isClosed=False):
  points = []
  for i in range(start, end+1):
    point = [landmarks.part(i).x, landmarks.part(i).y]
    points.append(point)

  points = np.array(points, dtype=np.int32)
  cv2.polylines(im, [points], isClosed, (255, 200, 0), 
                  thickness=2, lineType=cv2.LINE_8)

In [0]:
# Use this function for 68-points facial landmark detector model
def renderFace(im, landmarks):
    assert(landmarks.num_parts == 68)
    drawPolyline(im, landmarks, 0, 16)           # Jaw line
    drawPolyline(im, landmarks, 17, 21)          # Left eyebrow
    drawPolyline(im, landmarks, 22, 26)          # Right eyebrow
    drawPolyline(im, landmarks, 27, 30)          # Nose bridge
    drawPolyline(im, landmarks, 30, 35, True)    # Lower nose
    drawPolyline(im, landmarks, 36, 41, True)    # Left eye
    drawPolyline(im, landmarks, 42, 47, True)    # Right Eye
    drawPolyline(im, landmarks, 48, 59, True)    # Outer lip
    drawPolyline(im, landmarks, 60, 67, True)    # Inner lip

In [0]:
# Use this function for any model other than
# 68 points facial_landmark detector model
def renderFace2(im, landmarks, color=(0, 255, 0), radius=3):
  for p in landmarks.parts():
    cv2.circle(im, (p.x, p.y), radius, color, -1)

In [0]:
def writeLandmarksToFile(landmarks, landmarksFileName):
  with open(landmarksFileName, 'w') as f:
    for p in landmarks.parts():
      f.write("%s %s\n" %(int(p.x),int(p.y)))

  f.close()

In [0]:
def drawLandmarks(frameNo, im, faceRects):

  # Loop over all detected face rectangles
  for i in range(0, len(faceRects)):
      newRect = dlib.rectangle(int(faceRects[i].left()),
                         int(faceRects[i].top()),
                         int(faceRects[i].right()),
                         int(faceRects[i].bottom()))
      # For every face rectangle, run landmarkDetector
      landmarks = landmarkDetector(im, newRect)
      # Print number of landmarks
      #if i==0:
        #print("Number of landmarks",len(landmarks.parts()))
      # Store landmarks for current face
      landmarksAll.append(landmarks)

      # Next, we render the outline of the face using
      # detected landmarks.
      renderFace(im, landmarks)

      # The code below saves the landmarks to 
      # output/frame_0 … output/frame_N.txt.
      landmarksFileName = landmarksBasename +"_"+ str(frameNo)+ ".txt"
      #print("Saving landmarks to", landmarksFileName)
      # Write landmarks to disk
      writeLandmarksToFile(landmarks, landmarksFileName)

In [0]:
# Save output frames
def saveFrames(im, filename):
  outputFileName = OUTPUTPATH + filename
  print("Saving output image to", outputFileName)
  cv2.imwrite(outputFileName, im)

In [13]:
import os

if not os.path.exists(OUTPUTPATH): 
   os.makedirs(OUTPUTPATH) 
if not os.path.exists(LANDMARKSPATH): 
   os.makedirs(LANDMARKSPATH)

faceDetector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)
landmarksBasename = LANDMARKSPATH + "frame"

for i in range(0,142):
  imgname = srcframePrefix + str(i) + srcframeSuffix
  imgfile = DATA_PATH + imgname
  #print("Showing image :%s" % imgfile)
  im = cv2.imread(imgfile)
  faceRects = detectFace(im)
  
    #print(faceRects)
  # List to store landmarks of all detected faces
  landmarksAll = []
  
  drawLandmarks(i, im, faceRects)
  
  saveFrames(im, imgname)
  
  #plt.figure(figsize=(15,15))
  #plt.imshow(im[:,:,::-1])
  #plt.title("Facial Landmark detector")
  #plt.show()

Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame0.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame1.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame2.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame3.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame4.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame5.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame6.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame7.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame8.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame9.jpg
Saving output image to /content/drive/My Drive/EVA/Assignment23/LandmarksOutPut/frame10.jp